# Environemnt

In [1]:
import google.cloud.bigquery as bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snsn
import numpy as np
import importlib
import subprocess

In [2]:
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer,
    TfidfTransformer,
)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from skmultilearn.problem_transform import LabelPowerset
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    balanced_accuracy_score,
    cohen_kappa_score,
)

In [3]:
import src.utils.evaluation as evaluation
import src.model.sklearn_raphael.trainer.model as sklearn_v1

ModuleNotFoundError: No module named 'trainer'

In [ ]:
import importlib

importlib.reload(evaluation)
importlib.reload(sklearn_v1)

# Local Training

In [13]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PYTHONPATH}/src/model/sklearn_raphael/
    
python -m trainer.task \
--job-dir gs://${BUCKET_NAME}/stackoverflow/sklearn \
--project_id $PROJECT_ID \
--frac 0.05 \
--WE_min_df 1. \
--WE_max_df 1.

Process is terminated.


# Local Training (Fixed)

In [32]:
bucket_name = ! echo $BUCKET_NAME
project_id = ! echo $PROJECT_ID

In [39]:
command = '''python -m trainer.task \
--job-dir gs://${{{bucket_name}}}/stackoverflow/sklearn \
--project_id ${project_id} \
--frac 0.05 \
--WE_min_df 1. \
--WE_max_df 1.'''.format(bucket_name=bucket_name[0], project_id=project_id[0])

In [51]:
process = subprocess.call(command, shell=True)

In [52]:
process

1

# Training on the AI Platform (No Hyperparameter Tuning)

In [13]:
%%bash

RUNTIME_VERSION="1.14"
PYTHON_VERSION="3.5"
JOB_DIR="gs://${BUCKET_NAME}/stackoverflow/sklearn/${JOBNAME}"
JOB_NAME=stackoverflow_sklearn_$(date +"%Y%m%d_%H%M%S")
CONFIG="$DIR_PROJ/deployment/training/sklearn/custom.yaml"
gcloud ai-platform jobs submit training $JOB_NAME \
  --job-dir $JOB_DIR \
  --package-path $DIR_PROJ/src/model \
  --module-name model.sklearn_random_forest.task \
  --region us-central1 \
  --runtime-version=$RUNTIME_VERSION \
  --python-version=$PYTHON_VERSION \
  --config $CONFIG \
  -- \
  --project_id $PROJECT_ID \
  --frac 0.001

jobId: stackoverflow_sklearn_20191218_163349
state: QUEUED


Job [stackoverflow_sklearn_20191218_163349] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe stackoverflow_sklearn_20191218_163349

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs stackoverflow_sklearn_20191218_163349
